In [ ]:
#| default_exp core

# API

> Basic API for Caddy

# Caddy admin

In [ ]:
#| export
import os, subprocess, httpx, json
from fastcore.utils import *
from httpx import get as xget, post as xpost, patch as xpatch, put as xput, delete as xdelete, head as xhead

## Initial functions

In [ ]:
#| export
def get_id(path):
    "Get a ID full URL from a path"
    if path[0 ]!='/': path = '/'+path
    if path[-1]!='/': path = path+'/'
    return f'http://localhost:2019/id{path}'

In [ ]:
host = 'jph.answer.ai'

In [ ]:
get_id('jph.answer.ai')

'http://localhost:2019/id/jph.answer.ai/'

In [ ]:
#| export
def get_path(path):
    "Get a config full URL from a path"
    if path[0 ]!='/': path = '/'+path
    if path[-1]!='/': path = path+'/'
    return f'http://localhost:2019/config{path}'

In [ ]:
get_path('/apps/tls/automation/policies')

'http://localhost:2019/config/apps/tls/automation/policies/'

In [ ]:
#| export
def gid(path='/'):
    "Gets the id at `path`"
    response = xget(get_id(path))
    response.raise_for_status()
    return dict2obj(response.json())

In [ ]:
#| export
def gcfg(path='/', method='get'):
    "Gets the config at `path`"
    f = getattr(httpx, method)
    response = f(get_path(path))
    response.raise_for_status()
    return dict2obj(response.json())

In [ ]:
gcfg()

```json
{ 'apps': { 'tls': { 'automation': { 'policies': [{'issuers': [{'challenges': {'dns': {'provider': {'api_token': 'Q4WJIAN0l8CBCJg5ZNsPl38H69Y3I9tEomH12BrF', 'name': 'cloudflare'}}}, 'module': 'acme'}]}]}}}}
```

In [ ]:
#| export
def pid(d, path='/', method='post'):
    "Puts the config `d` into `path`"
    f = getattr(httpx, method)
    response = f(get_id(path), json=obj2dict(d))
    response.raise_for_status()
    return response.text or None

In [ ]:
#| export
def pcfg(d, path='/', method='post'):
    "Puts the config `d` into `path`"
    f = getattr(httpx, method)
    response = f(get_path(path), json=obj2dict(d))
    response.raise_for_status()
    return response.text or None

In [ ]:
# pcfg({})

In [ ]:
#| export
def nested_setdict(sd, value, *keys):
    "Returns `sd` updated to set `value` at the path `keys`"
    d = sd
    for key in keys[:-1]: d = d.setdefault(key, {})
    d[keys[-1]] = value
    return sd

In [ ]:
nested_setdict({'a':'b'}, {'c':'d'}, 'apps', 'http', 'servers', 'srv0')

{'a': 'b', 'apps': {'http': {'servers': {'srv0': {'c': 'd'}}}}}

In [ ]:
#| export
def path2keys(path):
    "Split `path` by '/' into a list"
    return path.strip('/').split('/')

In [ ]:
path2keys('/apps/tls/automation/policies')

['apps', 'tls', 'automation', 'policies']

In [ ]:
#| export
def keys2path(*keys):
    "Join `keys` into a '/' separated path"
    return '/'+'/'.join(keys)

In [ ]:
keys2path('apps', 'tls', 'automation', 'policies')

'/apps/tls/automation/policies'

In [ ]:
#| export
def nested_setcfg(value, *keys):
    d = nested_setdict(gcfg(), value, *keys)
    return pcfg(d)

In [ ]:
#| export
def init_path(path):
    sp = []
    for p in path2keys(path):
        sp.append(p)
        pcfg({}, keys2path(*sp))

## Acme setup

In [ ]:
cf_token = 'Q4WJIAN0l8CBCJg5ZNsPl38H69Y3I9tEomH12BrF'

In [ ]:
#| export
acme_path = '/apps/tls/automation'
def get_acme_config(token):
    prov = { "provider": { "name": "cloudflare", "api_token": token } }
    return {
        "module": "acme",
        "challenges": { "dns": prov }
    }

In [ ]:
#| export
def add_acme_config(cf_token):
    init_path(acme_path)
    val = [get_acme_config(cf_token)]
    pcfg([{'issuers':val}], acme_path+'/policies')

In [ ]:
# pcfg({})

In [ ]:
add_acme_config(cf_token)

In [ ]:
gcfg('/apps/tls/automation/policies')[0]

```json
{ 'issuers': [{'challenges': {'dns': {'provider': {'api_token': 'Q4WJIAN0l8CBCJg5ZNsPl38H69Y3I9tEomH12BrF', 'name': 'cloudflare'}}}, 'module': 'acme'}]}
```

## Route setup

In [ ]:
#| export
srvs_path = '/apps/http/servers'
rts_path = srvs_path+'/srv0/routes'

In [ ]:
#| export
def init_routes(srv_name='srv0'):
    # Create basic http server/routes config
    init_path(srvs_path)
    ir = {'listen': [':80', ':443'], 'routes': []}
    pcfg(ir, f"{srvs_path}/{srv_name}")

In [ ]:
init_routes()

In [ ]:
gcfg(srvs_path)

```json
{'srv0': {'listen': [':80', ':443'], 'routes': []}}
```

In [ ]:
#| export
def add_route(route):
    "Add `route` dict to config"
    return pcfg(route, rts_path)

In [ ]:
#| export
def add_reverse_proxy(from_host, to_url):
    "Create a reverse proxy handler"
    route = {
        "handle": [{
            "handler": "reverse_proxy",
            "upstreams": [{"dial": to_url}]
        }],
        "match": [{"host": [from_host]}],
        "@id": from_host,
        "terminal": True
    }
    add_route(route)

In [ ]:
host = "j.pla.sh"

In [ ]:
add_reverse_proxy(host, "localhost:5001")

In [ ]:
gcfg(srvs_path)

```json
{ 'srv0': { 'listen': [':80', ':443'],
            'routes': [{'@id': 'j.pla.sh', 'handle': [{'handler': 'reverse_proxy', 'upstreams': [{'dial': 'localhost:5001'}]}], 'match': [{'host': ['j.pla.sh']}], 'terminal': True}]}}
```

In [ ]:
gid(host)

```json
{ '@id': 'j.pla.sh',
  'handle': [{'handler': 'reverse_proxy', 'upstreams': [{'dial': 'localhost:5001'}]}],
  'match': [{'host': ['j.pla.sh']}],
  'terminal': True}
```

In [ ]:
#| export
def del_id(id):
    "Delete route for `id` (e.g. a host)"
    xdelete(get_id(id))

In [ ]:
# del_id(host)

In [ ]:
gcfg(rts_path)

(#0) []

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()